In [1]:
from ctgan import CTGANSynthesizer
from ctgan import load_demo
import warnings

warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
import copy

column_names = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num','marital-status', 'occupation', 'relationship', 'race', 'gender','capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']

train = pd.read_csv('adult_data.txt', sep=",\s", header=None, names = column_names, engine = 'python')
test = pd.read_csv('adult_test.txt', sep=",\s", header=None, names = column_names, engine = 'python')
test['income'].replace(regex=True,inplace=True,to_replace=r'\.',value=r'')


adult = pd.concat([test,train])
adult.reset_index(inplace = True, drop = True)

column_cat = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country','income']
column_int = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

for i in range(len(column_cat)):
    tmp_idx = adult[adult[column_cat[i]]=='?'].index
    adult = adult.drop(tmp_idx)

adult_all = copy.deepcopy(adult)
# for col in set(adult_all.columns) - set(adult_all.describe().columns):
#     adult_all[col] = adult_all[col].astype('category')

In [3]:
def drop_if(df, char):
    remove = []
    for c in df.columns:
        if char in c.split('_'):
            remove.append(c)
    return df.drop(columns = remove)

def extract_if(df, char):
    remove = []
    for c in df.columns:
        if char in c.split('_'):
            remove.append(c)
    return df[remove]

def normalize_and_split_dataset(data, label, tsize, target_i):
    xta, yta = data.loc[target_i], label.loc[target_i].values
    xtr, xte, ytr, yte = train_test_split(data, label.values, test_size  = tsize)
    scaler = StandardScaler()  
    scaler.fit(xtr)  
    xtr, xte, xta = scaler.transform(xtr), scaler.transform(xte), scaler.transform(xta)
    return (xtr, ytr), (xte, yte), (xta, yta), scaler

In [4]:
import copy
import numpy as np

n_target = 100
test_size = 0.25

column_cat = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country','income']
column_int = ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']

#================Data Prep===========================
    
adult_cat_1hot = pd.get_dummies(adult_all.select_dtypes('object'))
adult_non_cat = adult_all.select_dtypes(exclude = 'object')
adult_all_1hot = pd.concat([adult_non_cat, adult_cat_1hot], axis=1, join='inner')
adult_all_1hot = drop_if(adult_all_1hot, '?')

idxs = ['i%s' % i for i in range(len(adult_all_1hot))]
adult_all_1hot.index = idxs

#=====================================================

# Select Target data
target_indices = np.random.choice(adult_all_1hot.index, n_target)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler  

adult_data = drop_if(adult_all_1hot, 'income')
adult_label = extract_if(adult_all_1hot, 'income')

train_dataset, test_dataset, target_dataset, tr_scaler =  normalize_and_split_dataset(adult_data, adult_label, test_size, target_indices)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score


nn_tanh = MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), random_state=1, warm_start=True)
nn_relu_5_2 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_50_25 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(50, 25), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_200_100 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(200, 100), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_25_10 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(25, 10), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_log = MLPClassifier(activation = 'logistic', solver='adam', 
                    alpha=1e-4, hidden_layer_sizes=(5, 2),
                    learning_rate  = 'invscaling', 
                    random_state=1, warm_start = True)
nn_identity = MLPClassifier(activation = 'identity', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, warm_start = True)

tree_gini = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 0.05, min_samples_leaf = 0.001, max_features = None)
tree_entropy = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 0.05, min_samples_leaf = 0.001)

rf_gini = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)
rf_entropy = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

ada = AdaBoostClassifier(n_estimators=100)                     
log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = True, solver = 'liblinear')


model_names = ['nn_tanh','nn_relu_5_2', 'nn_relu_50_25', 'nn_relu_200_100', 'nn_relu_25_10','nn_log', 'nn_identity',
               'tree_gini', 'tree_entropy', 
               'rf_gini', 'rf_entropy', 
               'gb', 'ada', 
               'log_reg']
org_models = [nn_tanh,nn_relu_5_2, nn_relu_50_25, nn_relu_200_100, nn_relu_25_10, nn_log, nn_identity,
          tree_gini, tree_entropy, 
          rf_gini, rf_entropy, 
          gb, ada, 
          log_reg]

n_models = len(org_models)

In [7]:
result = []

models = copy.deepcopy(org_models)
x_tr, y_tr = train_dataset[0], train_dataset[1][:,1]
x_te, y_te = test_dataset[0], test_dataset[1][:,1]
x_ta, y_ta = target_dataset[0], target_dataset[1][:,1]

for model, name in zip(models, model_names):
    
    print(name)
    model.fit(x_tr, y_tr)
    
    pred = model.predict(x_tr)
    tr_acc = accuracy_score(y_tr, pred)
    
    pred = model.predict(x_te)
    te_acc = accuracy_score(y_te, pred)
    
    pred = model.predict(x_ta)
    ta_acc = accuracy_score(y_ta, pred)
    
    result.append([tr_acc, te_acc, ta_acc, 0])

nn_tanh
nn_relu_5_2
nn_relu_50_25
nn_relu_200_100
nn_relu_25_10
nn_log
nn_identity
tree_gini
tree_entropy
rf_gini
rf_entropy
gb
ada
log_reg


In [83]:
from sklearn.metrics import log_loss

def cross_entropy(y1, y2):
    def cce(y_true, y_pred):
        one_hot = np.zeros(2)
        one_hot[y_true] = 1
        return log_loss([one_hot], [y_pred])
    return np.array([cce(y1[i], y2[i]) for i in range(len(y1))])

labels_train = np.ones(len(y_tr))
labels_test = np.zeros(len(y_te))

In [90]:
tmp = []
switch = False

for x1 in x_ta:
    for x2 in x_tr:
        if np.sum(x1!=x2) == 0:
            tmp.append(1)
            switch = True
            break
    if switch is not True:
        tmp.append(0)
    else:
        switch = False

labels_target = np.array(tmp)

In [176]:
def delete(x, f):
    tmp = []
    switch = False
    f_t = copy.deepcopy(f)

    for i, f1 in enumerate(x):
        for j, f2 in enumerate(f_t):
            if np.sum(f1!=f2) == 0:
                switch = True
                break
        if switch is not True:
            tmp.append(i)
        else:
            f_t = np.delete(f_t, j, axis=0)
            switch = False
    return tmp


In [177]:
attack_result = []
before_attack_models = []

for model, name in zip(models, model_names):
    
    print(name)
    
    prob_train = model.predict_proba(x_tr)
    prob_test = model.predict_proba(x_te)
    prob_target = model.predict_proba(x_ta)

    loss_train = cross_entropy(y_tr, prob_train)
    loss_test = cross_entropy(y_te, prob_test)
    loss_target = cross_entropy(y_ta, prob_target)

    feature_train = np.c_[prob_train, loss_train]
    feature_test = np.c_[prob_test, loss_test]
    feature_target = np.c_[prob_target, loss_target]

    x_mia = np.r_[feature_train, feature_test]
    y_mia = np.r_[labels_train, labels_test]

    x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)
    
    a_idxs = delete(x_mia_tr, feature_target)
    x_mia_tr = x_mia_tr[a_idxs]
    y_mia_tr = y_mia_tr[a_idxs]
    
    attack_model = MLPClassifier(hidden_layer_sizes=(5, 5), random_state=1, max_iter=300)
    attack_model.fit(x_mia_tr, y_mia_tr)
    
    pred = attack_model.predict(x_mia_te)
    prob = attack_model.predict_proba(x_mia_te)
    mia_conf = np.abs(prob[:,1]-0.5).mean()
    mia_acc = accuracy_score(y_mia_te, pred)
    
    pred = attack_model.predict(feature_target)
    prob = attack_model.predict_proba(feature_target)
    ta_mia_conf = np.abs(prob[:,1]-0.5).mean()
    ta_mia_acc = accuracy_score(labels_target, pred)
    
    attack_result.append([mia_acc, mia_conf, ta_mia_acc, ta_mia_conf])
    before_attack_models.append(attack_model)
    

nn_tanh
33840
nn_relu_5_2
33841
nn_relu_50_25
33836
nn_relu_200_100
33838
nn_relu_25_10
33842
nn_log
33844
nn_identity
33848
tree_gini
33816
tree_entropy
33816
rf_gini
33822
rf_entropy
33822
gb
33828
ada
33829
log_reg
33848


In [178]:
round(pd.DataFrame(attack_result, index=model_names, columns=['mia acc', 'mia conf', 'ta mia acc', 'ta mia conf']),4)

,mia acc,mia conf,ta mia acc,ta mia conf
nn_tanh,0.7488,0.2456,0.80,0.2460
nn_relu_5_2,0.7508,0.2521,0.81,0.2525
nn_relu_50_25,0.7551,0.2613,0.80,0.2582
nn_relu_200_100,0.7722,0.2744,0.80,0.2658
nn_relu_25_10,0.7495,0.2531,0.81,0.2556
nn_log,0.7456,0.2506,0.81,0.2516
nn_identity,0.7533,0.2430,0.81,0.2426
tree_gini,0.7473,0.2554,0.81,0.2557
tree_entropy,0.7453,0.2456,0.81,0.2458
rf_gini,0.7510,0.2508,0.81,0.2509


In [10]:
from tqdm import tqdm

def label_matrix(surrogate_models, data, a):
    def step(y, th):
        y[np.abs(y - 0.5) < th] = -1
        y[y > 0.5] = 1
        y[np.abs(y) < 0.5] = 0
        return y
    
    matrix = []
    for model in tqdm(surrogate_models):
        try:
            prob = model.predict_proba(data)[:,1]
            pred = step(prob, a)
        except:
            pred = model.predict(data)
        matrix.append(pred)        
    return np.array(matrix)

In [11]:
from snorkel.labeling.model import LabelModel

alpha = 0.1
y_te = test_dataset[1][:,1]
th = 0.95

midxs = [i for i in range(13)]#[1,9,10,11,12]
L_train = label_matrix([models[i] for i in midxs], train_dataset[0], alpha)
L_test = label_matrix([models[i] for i in midxs], test_dataset[0], alpha)

label_model = LabelModel(cardinality=2, verbose=True)
L_all = np.concatenate((L_train, L_test), axis=1)
label_model.fit(L_all.T, n_epochs=500, log_freq=50, seed=25)
sn_prob = label_model.predict_proba(L=L_test.T)

p_idxs = np.max(sn_prob, axis=1)>=th
pred = np.argmax(sn_prob[p_idxs], axis=1)

100%|██████████| 5/5 [00:00<00:00,  6.19it/s]


In [12]:
import pandas as pd

def abnormal_examples(df, col1, col2, th=1e-4):
    abnormal = []
    confusion_matrix = pd.crosstab(df[col1], df[col2])/len(df) <= th
    val1, val2 = confusion_matrix.index, confusion_matrix.columns
    for v2 in val2:
        tmp = val1[confusion_matrix[v2]]
        abnormal.extend([{col1:t, col2:v2} for t in tmp])        
    return abnormal

In [13]:
# filter_list = []

# filter_list.extend(abnormal_examples(adult_all, 'relationship', 'gender'))
# filter_list.extend(abnormal_examples(adult_all, 'education', 'educational-num'))
# filter_list.extend(abnormal_examples(adult_all, 'relationship', 'marital-status'))

In [14]:
filter_list = []
candidate = ['workclass', 'education', 'educational-num', 'marital-status', 'occupation', 'relationship', 'race', 'gender']

for i, c1 in enumerate(candidate):
    for c2 in candidate[i+1:]:
        filter_list.extend(abnormal_examples(adult_all, c1, c2))

In [15]:
ctgan = CTGANSynthesizer(epochs=100)
ctgan.fit(adult, column_cat)

In [135]:
# Synthetic copy
n_gen = 10000000
samples = ctgan.sample(n_gen)

In [136]:
samples

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,49,Private,25760,Masters,14,Never-married,Sales,Not-in-family,White,Female,0,1,48,Italy,<=50K
1,59,Private,133894,5th-6th,4,Divorced,Sales,Unmarried,White,Female,-2,0,50,Mexico,<=50K
2,55,Private,165178,Assoc-voc,10,Divorced,Sales,Unmarried,White,Female,6,0,34,United-States,<=50K
3,47,Self-emp-not-inc,169565,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,14352,0,69,United-States,>50K
4,54,Private,155750,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,-27,0,40,Germany,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,44,Private,173884,10th,6,Married-civ-spouse,Sales,Husband,White,Male,-4,0,40,China,>50K
9999996,54,Private,138180,Some-college,10,Married-civ-spouse,Sales,Wife,White,Female,0,0,34,United-States,>50K
9999997,17,Private,214883,11th,6,Never-married,Priv-house-serv,Own-child,White,Female,0,-1,40,United-States,<=50K
9999998,36,Private,120136,Masters,14,Never-married,Exec-managerial,Husband,Asian-Pac-Islander,Male,2,0,48,Taiwan,>50K


In [137]:
from copy import deepcopy

def filter_table(df, d):
    cond1 = lambda x: (x[list(d.keys())[0]] == list(d.values())[0]).astype(int)
    cond2 = lambda x: (x[list(d.keys())[1]] == list(d.values())[1]).astype(int)
    return df[(cond1(df)+cond2(df)) == 2]

samples_clean = deepcopy(samples)

for f in filter_list:
    abnormals = filter_table(samples_clean, f)
    samples_clean = samples_clean.drop(abnormals.index)

In [138]:
samples_clean

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,49,Private,25760,Masters,14,Never-married,Sales,Not-in-family,White,Female,0,1,48,Italy,<=50K
3,47,Self-emp-not-inc,169565,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,14352,0,69,United-States,>50K
4,54,Private,155750,7th-8th,4,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,-27,0,40,Germany,<=50K
8,72,Private,85410,HS-grad,9,Married-civ-spouse,Other-service,Husband,Black,Male,8,0,46,United-States,<=50K
9,52,Private,215348,7th-8th,4,Married-civ-spouse,Machine-op-inspct,Husband,Other,Male,9,1,40,Thailand,>50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999992,44,Self-emp-not-inc,313931,Prof-school,15,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,>50K
9999993,42,Private,318569,7th-8th,4,Divorced,Transport-moving,Unmarried,Asian-Pac-Islander,Male,-16,0,39,El-Salvador,<=50K
9999995,44,Private,173884,10th,6,Married-civ-spouse,Sales,Husband,White,Male,-4,0,40,China,>50K
9999996,54,Private,138180,Some-college,10,Married-civ-spouse,Sales,Wife,White,Female,0,0,34,United-States,>50K


In [139]:
def flatten_data(data):
    data_cat = pd.get_dummies(data.select_dtypes('object'))
    data_int = data.select_dtypes(exclude = 'object')
    data_1hot = pd.concat([data_cat, data_int], axis=1, join='inner')
    data_1hot = drop_if(data_1hot, 'income')
    data_1hot.index = np.arange(len(data_1hot))
    all_column = drop_if(adult_all_1hot, 'income').columns
    d = pd.DataFrame(0, index=np.arange(len(data_1hot)), columns=all_column)
    for c in data_1hot.columns:
        d[c] = data_1hot[c]
    return d

candidates = flatten_data(samples_clean)
candidates = tr_scaler.transform(candidates)

In [426]:
tmp = copy.deepcopy(adult_all_1hot)
#tmp = drop_if(tmp, 'income')

In [427]:
alpha = 0.4
r = lambda x : round(x+0.01)
a1, a2 = tmp.loc['i0'], tmp.loc['i2']
r1 = a1*alpha+a2*(1-alpha)
r2 = r(r1)

In [428]:
np.sum(tmp.loc['i0'][-2:]==tmp.loc['i2'][-2:])

0

In [512]:
tmp_index = np.array(tmp.index)
r = lambda x : round(x+0.001)
linear_cands = []
linear_reals = []

for ti in tqdm(target_indices):
    tmp_target, tmp_target_label = tmp.loc[ti][:-2], tmp.loc[ti][-2:]
    tmp_target_norm = tr_scaler.transform([tmp_target])
    tmp_cand_idx, tmp_dist = [], []
    for ii in tmp_index:
        tmp_cand, tmp_cand_label = tmp.loc[ii][:-2], tmp.loc[ii][-2:]
        if np.sum(tmp_cand_label==tmp_target_label) == 0:
            tmp_cand_norm = tr_scaler.transform([tmp_cand])
            tmp_dist.append(np.linalg.norm(tmp_target_norm-tmp_cand_norm))
            tmp_cand_idx.append(ii)
    sorted_idx = np.argsort(tmp_dist)
    tmp_cand_idx = np.array(tmp_cand_idx)
    nearest = tmp.loc[tmp_cand_idx[sorted_idx][1]][:-2]
    nearest_norm = tr_scaler.transform([nearest])
    n_dist = np.linalg.norm(tmp_target_norm-nearest_norm)
    
    lin = []
    for li in range(1,30):
        tmp_linear_num = (1-li/30)*tmp_target+(li/30)*nearest
        tmp_linear = r(tmp_linear_num)
        tmp_linear_norm = tr_scaler.transform([tmp_linear])
        #print(n_dist, np.linalg.norm(tmp_target_norm-tmp_linear_norm), np.linalg.norm(nearest_norm-tmp_linear_norm))
        lin.append(tmp_linear)
    
    lin_norm = tr_scaler.transform(lin)
    linear_cands.append(lin_norm)      
    linear_reals.append(lin)

100%|██████████| 100/100 [59:17<00:00, 35.58s/it]


In [434]:
x_ta, y_ta = target_dataset[0], target_dataset[1][:,1]

n_cand = len(samples_clean)//1000
generated = []
generated_real = []

for i in range(len(y_ta)):
    tmp_dist = []
    for c in candidates:
        tmp_dist.append(np.linalg.norm(c-x_ta[i]))
    
    generated.extend(candidates[np.argsort(tmp_dist)[:n_cand]])
    generated_real.extend(samples_clean.values[np.argsort(tmp_dist)[:n_cand]])

generated = np.array(generated)

In [ ]:
generated.shape

In [435]:
L_gen = label_matrix([models[i] for i in midxs], generated, alpha)

100%|██████████| 5/5 [00:27<00:00,  5.46s/it]


In [505]:
probs = label_model.predict_proba(L_gen.T)
count = 0
threshold = 0.95
dists = []
lins = []
disinfos_gan = []
examples_gan = []

disinfos_wm = []
examples_wm = []

disinfos_optim = []
examples_optim = []
bases = []

for ti in range(len(y_ta)):
    conf_idx = probs[:,y_ta[ti]]<(1-threshold)
    dist1 = []
    for g in generated[conf_idx]:
        dist1.append(np.linalg.norm(g-x_ta[ti]))
    
    disinfo = generated[conf_idx][np.argmin(dist1)]
    disinfos_gan.append(disinfo)
    conf_idx_convert = np.arange(len(conf_idx))[conf_idx]
    example = [generated_real[c] for c in conf_idx_convert][np.argmin(dist1)]
    examples_gan.append(example)
    
    dist2 = []
    for g in train_dataset[0][train_dataset[1][:,1]!=y_ta[ti]]:
        dist2.append(np.linalg.norm(g-x_ta[ti]))
        
    base = train_dataset[0][train_dataset[1][:,1]!=y_ta[ti]][np.argmin(dist2)]
    bases.append(base)
    
    L_lin = label_matrix([models[i] for i in midxs], linear_cands[ti], alpha)
    lin_probs = label_model.predict_proba(L_lin.T)
    
    lin_conf_idx = lin_probs[:,y_ta[ti]]<(1-threshold)
    dist3 = []
    for g in linear_cands[ti][lin_conf_idx]:
        dist3.append(np.linalg.norm(g-x_ta[ti]))
    
    if len(dist3) == 0:
        disinfos_wm.append(base)
        examples_wm.append('base')
        dist3.append(100)
    else:
        disinfos_wm.append(linear_cands[ti][lin_conf_idx][np.argmin(dist3)])
        conf_idx_convert = np.arange(len(conf_idx))[conf_idx]
        examples_wm.append([linear_reals[ti][c] for c in lin_conf_idx][np.argmin(dist3)])
        
    if min([min(dist1),min(dist2),min(dist3)]) == min(dist1):
        count += 1
        
    didx = np.argmin([min(dist1),min(dist2),min(dist3)])
    disinfos_optim.append(np.array([disinfos_gan[-1], base, disinfos_wm[-1]])[didx])
    examples_optim.append(np.array([examples_gan[-1], 'base', examples_wm[-1]])[didx])

    print(min(dist1), min(dist2), min(dist3), 'gan' if min([min(dist1),min(dist2),min(dist3)]) == min(dist1) else 'x')
    dists.append([min(dist1), min(dist2), min(dist3)])
    
#     if min([min(dist1),min(dist2)]) == min(dist1):
#         count += 1

#     print(min(dist1), min(dist2), 'gan' if min([min(dist1),min(dist2)]) == min(dist1) else 'x')
#     dists.append([min(dist1), min(dist2)])

100%|██████████| 5/5 [00:00<00:00, 161.64it/s]


1.0135190772064218 0.40219472039735843 7.215548469940634 x


100%|██████████| 5/5 [00:00<00:00, 158.92it/s]


7.36507290054686 7.243502371330123 20.572431438565836 x


100%|██████████| 5/5 [00:00<00:00, 155.47it/s]


...

7.1666142452598836 7.621981039140681 18.59584775539023 gan


100%|██████████| 5/5 [00:00<00:00, 28.49it/s]


3.37635765028699 5.400106450657698 8.997974131192368 gan


100%|██████████| 5/5 [00:00<00:00, 28.45it/s]


5.1717185054240895 1.5277395909531455 5.893950333527993 x


In [447]:
adult.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

In [453]:
adult_all_1hot

,age,fnlwgt,educational-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,income_<=50K,income_>50K
i0,25,226802,7,0,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
i1,38,89814,9,0,0,50,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
i2,28,336951,12,0,0,40,0,1,0,0,...,0,0,0,0,0,1,0,0,0,1
i3,44,160323,10,7688,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
i4,34,198693,6,0,0,30,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
i45217,27,257302,12,0,0,38,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
i45218,40,154374,9,0,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
i45219,58,151910,9,0,0,40,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
i45220,22,201490,9,0,0,20,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0


In [508]:
adult_tmp = copy.deepcopy(adult)
adult_tmp.index = adult_all_1hot.index

In [509]:
adult_tmp.loc[target_indices[np.argmin(dists,axis=1)==2]]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
i45080,35,Private,379959,HS-grad,9,Divorced,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K
i30694,20,Private,210338,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,15,United-States,<=50K
i20100,43,Private,131435,Bachelors,13,Divorced,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K
i33561,44,Local-gov,231793,Doctorate,16,Married-spouse-absent,Prof-specialty,Unmarried,White,Female,0,0,38,United-States,<=50K


In [472]:
idx = np.arange(n_target)[np.argmin(dists,axis=1)==2]
pd.DataFrame([examples_optim[i] for i in idx], columns=adult.columns)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,37.0,NaN,21255.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,14637.0,0.0,45.0,NaN,NaN
1,37.0,NaN,21255.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,14637.0,0.0,45.0,NaN,NaN
2,37.0,NaN,21255.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,14637.0,0.0,45.0,NaN,NaN
3,37.0,NaN,21255.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,14637.0,0.0,45.0,NaN,NaN


In [510]:
def shorten(a, col_int, col_cat):
    new = pd.DataFrame(0, columns=adult.columns, index=[0])
    for c in col_int:
        new[c] = a[c]
    
    for c in col_cat:
        x = extract_if(a, c)
        new[c] = x.columns[np.argmax(x)].split('_')[-1]
    return new

for ii in range(4):
    a = copy.deepcopy([examples_optim[i] for i in idx][ii])
    a = a.to_frame().T
    print(shorten(a, column_int, column_cat[:-1]))

    age workclass   fnlwgt education  educational-num      marital-status  \
0  37.0   Private  21255.0   HS-grad              9.0  Married-civ-spouse   

     occupation relationship   race gender  capital-gain  capital-loss  \
0  Craft-repair      Husband  White   Male       14637.0           0.0   

   hours-per-week native-country  income  
0            45.0  United-States       0  
    age workclass   fnlwgt education  educational-num      marital-status  \
0  37.0   Private  21255.0   HS-grad              9.0  Married-civ-spouse   

     occupation relationship   race gender  capital-gain  capital-loss  \
0  Craft-repair      Husband  White   Male       14637.0           0.0   

   hours-per-week native-country  income  
0            45.0  United-States       0  
    age workclass   fnlwgt education  educational-num      marital-status  \
0  37.0   Private  21255.0   HS-grad              9.0  Married-civ-spouse   

     occupation relationship   race gender  capital-gain  capital

In [507]:
disinfos_wm

[array([-0.34128189, -1.28717273, -0.438829  ,  0.57490621, -0.21850024,
         0.00393781, -0.17952876, -0.27072336,  0.59546707, -0.19193166,
        -0.3012009 , -0.21224603, -0.02103485, -0.16585681, -0.19333103,
        -0.11211173, -0.06991955, -0.10047983, -0.13724085, -0.12006857,
        -0.18394367, -0.21443656, -0.44770402, -0.11331896,  1.4304422 ,
        -0.24204307, -0.04030245, -0.13282629, -0.5284598 , -0.40212656,
        -0.02715988,  1.07632129, -0.11224648, -0.69222456, -0.18109768,
        -0.17167808, -0.3745178 , -0.01881331,  2.57224384, -0.39077317,
        -0.18299929, -0.2206803 , -0.26298927, -0.34534918, -0.06863042,
        -0.39377543, -0.14986453, -0.36602329, -0.18325727, -0.23240494,
         1.19788144, -0.58951734, -0.17794774, -0.41421927, -0.34775091,
        -0.22053332, -0.09957776, -0.17113126, -0.32288099, -0.08823325,
         0.40415283, -0.69660882,  0.69660882, -0.02489097, -0.06082113,
        -0.05100388, -0.04209765, -0.05492269, -0.0

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [506]:
count = [0,0,0]
count2 = 0

for d1, d2, d3 in dists:
    i = np.argmin([d1, d2, d3])
    count[i] += 1
    if d1 < d2:
        count2 += 1
print(count, count2)

[44, 52, 4] 44


In [149]:
L_target = label_matrix([models[i] for i in midxs], x_ta, alpha)
pred = np.argmax(label_model.predict_proba(L_target.T), axis=1)
accuracy_score(y_ta, pred)

100%|██████████| 5/5 [00:00<00:00, 142.13it/s]


0.85

In [150]:
L_base = label_matrix([models[i] for i in midxs], np.array(bases), alpha)
#pred = np.argmax(label_model.predict_proba(L_base.T), axis=1)
np.max(label_model.predict_proba(L_base.T), axis=1)

100%|██████████| 5/5 [00:00<00:00, 157.44it/s]


array([0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.87742751, 0.5       ,
       0.5       , 0.72895136, 1.        , 0.98056944, 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 0.5       ,
       0.5       , 0.98056944, 0.5       , 0.5       , 0.72895136,
       0.87742751, 0.94940475, 0.5       , 0.5       , 0.98056944,
       1.        , 0.5       , 1.        , 0.5       , 1.        ,
       1.        , 0.87577279, 0.5       , 0.5       , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.72895136, 0.5       ,
       0.5       , 0.87577279, 0.5       , 0.98056944, 0.5       ,
       0.5       , 0.98056944, 0.5       , 1.        , 0.5       ,
       0.5       , 0.5       , 0.5       , 0.5       , 1.        ,
       0.5       , 0.72690638, 0.5       , 0.5       , 0.5       ,
       0.87742751, 0.5       , 0.98056944, 0.98056944, 0.87742751,
       0.87742751, 0.98056944, 0.5       , 0.5       , 0.5    

In [385]:
nn_tanh = MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), random_state=1, warm_start=True)
nn_relu_5_2 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_50_25 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(50, 25), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_200_100 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(200, 100), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_25_10 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(25, 10), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_log = MLPClassifier(activation = 'logistic', solver='adam', 
                    alpha=1e-4, hidden_layer_sizes=(5, 2),
                    learning_rate  = 'invscaling', 
                    random_state=1, warm_start = True)
nn_identity = MLPClassifier(activation = 'identity', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, warm_start = True)

tree_gini = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 0.05, min_samples_leaf = 0.001, max_features = None)
tree_entropy = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 0.05, min_samples_leaf = 0.001)

rf_gini = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)
rf_entropy = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

ada = AdaBoostClassifier(n_estimators=100)                     
log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = True, solver = 'liblinear')


poisoned_models = [nn_tanh,nn_relu_5_2, nn_relu_50_25, nn_relu_200_100, nn_relu_25_10, nn_log, nn_identity,
          tree_gini, tree_entropy, 
          rf_gini, rf_entropy, 
          gb, ada, 
          log_reg]

tidx = 2
n_repeat = 200

poison_x_repeat = np.repeat(disinfos_optim[tidx*10:(tidx+1)*10], n_repeat, axis=0)
poison_y_repeat = np.repeat(target_dataset[1][:,0][tidx*10:(tidx+1)*10], n_repeat, axis=0)

poisoned_x = np.concatenate((train_dataset[0], poison_x_repeat), axis=0)
poisoned_y = np.concatenate((train_dataset[1][:,1], poison_y_repeat), axis=0)

In [386]:
poisoned_result = []
#conf_ds = []

for i, (model, name) in enumerate(zip(poisoned_models, model_names)):
    
    print(name)
    model.fit(poisoned_x, poisoned_y)
    
    pred1 = models[i].predict(train_dataset[0])
    pred2 = model.predict(train_dataset[0])
    tr_acc1 = accuracy_score(train_dataset[1][:,1], pred1)
    tr_acc2 = accuracy_score(train_dataset[1][:,1], pred2)
    
    pred1 = models[i].predict(test_dataset[0])
    pred2 = model.predict(test_dataset[0])
    te_acc1 = accuracy_score(test_dataset[1][:,1], pred1)
    te_acc2 = accuracy_score(test_dataset[1][:,1], pred2)
    
    pred1 = models[i].predict(target_dataset[0][tidx*10:(tidx+1)*10])
    pred2 = model.predict(target_dataset[0][tidx*10:(tidx+1)*10])
    #ti = target_dataset[1][:,1][tidx*10:(tidx+1)*10]
    ta_acc1 = accuracy_score(ti, pred1)
    ta_acc2 = accuracy_score(ti, pred2)
    
    try:
        prob1 = models[i].predict_proba(target_dataset[0][tidx*10:(tidx+1)*10])
        ti = models[i].predict(target_dataset[0][tidx*10:(tidx+1)*10]).astype(int)
        prob2 = model.predict_proba(target_dataset[0][tidx*10:(tidx+1)*10])
        conf_d = np.mean(prob2[np.arange(len(ti)), ti]-prob1[np.arange(len(ti)), ti])
    except:
        conf_d = ta_acc2-ta_acc1
    
    poisoned_result.append([tr_acc2-tr_acc1, te_acc2-te_acc1, ta_acc2-ta_acc1, conf_d])
    #conf_ds.append(conf_d)

nn_tanh
nn_relu_5_2
nn_relu_50_25
nn_relu_200_100
nn_relu_25_10
nn_log
nn_identity
tree_gini
tree_entropy
rf_gini
rf_entropy
gb
ada
log_reg


In [319]:
np.random.choice(np.arange(len(x_tr)), len(x_te))

array([26982, 17405, 18652, ..., 21606,  8458,  1074])

In [393]:
from sklearn.metrics import f1_score

before_attack_result = []
after_attack_result = []

seed = 101

nn_tanh = MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), max_iter=1000, random_state=seed, warm_start=True)
nn_relu_5_2 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=seed, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_50_25 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(50, 25), random_state=seed, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_200_100 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(200, 100), max_iter=2000, random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_25_10 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(25, 10), random_state=seed, learning_rate  = 'invscaling',
                    warm_start = True)
nn_log = MLPClassifier(activation = 'logistic', solver='adam', 
                    alpha=1e-4, hidden_layer_sizes=(5, 2),
                    learning_rate  = 'invscaling', 
                    random_state=seed, warm_start = True)
nn_identity = MLPClassifier(activation = 'identity', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=seed, warm_start = True)

tree_gini = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 0.05, min_samples_leaf = 0.001, max_features = None,random_state=seed)
tree_entropy = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 0.05, min_samples_leaf = 0.001,random_state=seed)

rf_gini = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001,random_state=seed)
rf_entropy = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001,random_state=seed)

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=seed)

ada = AdaBoostClassifier(n_estimators=100)                     
log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = True, solver = 'liblinear')


mia_models = [nn_tanh,nn_relu_5_2, nn_relu_50_25, nn_relu_200_100, nn_relu_25_10, nn_log, nn_identity,
          tree_gini, tree_entropy, 
          rf_gini, rf_entropy, 
          gb, ada, 
          log_reg]

i = 0
print(model_names[i])

for ba_model in mia_models:
    
    print(ba_model)
    
    sample_idx = np.random.choice(np.arange(len(x_tr)), len(x_te))
    
    prob_train = models[i].predict_proba(x_tr[sample_idx])
    prob_test = models[i].predict_proba(x_te)
    prob_target = models[i].predict_proba(x_ta)

    loss_train = cross_entropy(y_tr[sample_idx], prob_train)
    loss_test = cross_entropy(y_te, prob_test)
    loss_target = cross_entropy(y_ta, prob_target)

    feature_train = np.c_[prob_train, loss_train]
    feature_test = np.c_[prob_test, loss_test]
    feature_target = np.c_[prob_target, loss_target]

    x_mia = np.r_[feature_train, feature_test]
    y_mia = np.r_[labels_train[sample_idx], labels_test]

    x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)
    
#     a_idxs = delete(x_mia_tr, feature_target)
#     x_mia_tr = x_mia_tr[a_idxs]
#     y_mia_tr = y_mia_tr[a_idxs]
    
    #ba_model =  MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), max_iter=1000, random_state=1, warm_start=True)
    ba_model.fit(x_mia_tr, y_mia_tr)
    
    pred = ba_model.predict(x_mia_te)
    print(np.sum(pred)/len(pred))
    prob = ba_model.predict_proba(x_mia_te)
    mia_conf1 = np.max(prob, axis=1).mean()
    mia_f1_1 = f1_score(y_mia_te, pred)
    mia_acc1 = accuracy_score(y_mia_te, pred)
    
    pred = ba_model.predict(feature_target[tidx*10:(tidx+1)*10])
    prob = ba_model.predict_proba(feature_target[tidx*10:(tidx+1)*10])
    ta_mia_conf1 = np.max(prob, axis=1).mean()
    ta_mia_acc1 = accuracy_score(labels_target[tidx*10:(tidx+1)*10], pred)
    
    before_attack_result.append([mia_f1_1, mia_acc1, mia_conf1, ta_mia_acc1, ta_mia_conf1])
    print(before_attack_result[-1])
    

    #=========================================================================
    
    prob_train = poisoned_models[i].predict_proba(x_tr[sample_idx])
    prob_test = poisoned_models[i].predict_proba(x_te)
    prob_target = poisoned_models[i].predict_proba(x_ta)

    loss_train = cross_entropy(y_tr[sample_idx], prob_train)
    loss_test = cross_entropy(y_te, prob_test)
    loss_target = cross_entropy(y_ta, prob_target)

    feature_train = np.c_[prob_train, loss_train]
    feature_test = np.c_[prob_test, loss_test]
    feature_target = np.c_[prob_target, loss_target]

    x_mia = np.r_[feature_train, feature_test]
    y_mia = np.r_[labels_train[sample_idx], labels_test]

    x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)
    
#     a_idxs = delete(x_mia_tr, feature_target)
#     x_mia_tr = x_mia_tr[a_idxs]
#     y_mia_tr = y_mia_tr[a_idxs]
    
    attack_model =  MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), max_iter=1000, random_state=1, warm_start=True)
    attack_model.fit(x_mia_tr, y_mia_tr)
    
    pred = ba_model.predict(x_mia_te)
    print(np.sum(pred)/len(pred))
    prob = ba_model.predict_proba(x_mia_te)
    mia_conf2 = np.max(prob, axis=1).mean()
    mia_f1_2 = f1_score(y_mia_te, pred)
    mia_acc2 = accuracy_score(y_mia_te, pred)
    
    pred = ba_model.predict(feature_target[tidx*10:(tidx+1)*10])
    prob = ba_model.predict_proba(feature_target[tidx*10:(tidx+1)*10])
    ta_mia_conf2 = np.max(prob, axis=1).mean()
    ta_mia_acc2 = accuracy_score(labels_target[tidx*10:(tidx+1)*10], pred)
    
    after_attack_result.append([mia_f1_2, mia_acc2, mia_conf2, ta_mia_acc2, ta_mia_conf2])
    print(after_attack_result[-1])
    

nn_tanh
MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=(10, 2),
              max_iter=1000, random_state=101, solver='lbfgs', warm_start=True)
0.6212630461701751
[0.5598991172761665, 0.5061029541836193, 0.5156719095377326, 0.7, 0.511815511318169]
0.6009198655581107
[0.5556450314060235, 0.5119405625331682, 0.5152205882854255, 0.7, 0.5134524511206006]
MLPClassifier(alpha=0.1, hidden_layer_sizes=(5, 2), learning_rate='invscaling',
              random_state=101, warm_start=True)
0.8676808774102247
[0.6372854914196568, 0.5064567486290465, 0.5159201874690608, 0.9, 0.5114559696978438]
0.8588360162745445
[0.638420295540735, 0.5108791791968866, 0.5160536114065164, 0.8, 0.5143319625178142]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(50, 25),
              learning_rate='invscaling', random_state=101, warm_start=True)
0.7148416769856714
[0.5910144927536232, 0.5007960375022112, 0.516960390980629, 0.7, 0.5153603616708498]
0.6745091102069698
[0.5999702248027393, 0.52467716256854

In [394]:
[model_names[i] for i in [0,2,3,6,10,11,12,13]]

['nn_tanh',
 'nn_relu_50_25',
 'nn_relu_200_100',
 'nn_identity',
 'rf_entropy',
 'gb',
 'ada',
 'log_reg']

In [321]:
after_attack_result1 = np.array(after_attack_result)
before_attack_result1 = np.array(before_attack_result)

round(pd.DataFrame(after_attack_result1-before_attack_result1, index=model_names),4)

,0,1,2,3,4
nn_tanh,0.0150,0.0157,0.0002,-0.1,-0.0146
nn_relu_5_2,-0.0094,-0.0108,-0.0009,-0.1,-0.0088
nn_relu_50_25,-0.0065,-0.0057,0.0002,-0.1,-0.0090
nn_relu_200_100,0.0139,0.0117,-0.0008,0.0,-0.0164
nn_relu_25_10,0.0118,0.0152,-0.0002,0.1,-0.0131
nn_log,-0.0020,0.0004,-0.0001,0.0,-0.0009
nn_identity,-0.0245,-0.0129,-0.0009,0.1,-0.0071
tree_gini,-0.0222,0.0042,-0.0044,-0.3,-0.0609
tree_entropy,-0.0105,-0.0083,-0.0132,-0.2,-0.0192
rf_gini,-0.0056,-0.0025,-0.0031,-0.4,-0.0312


In [322]:
round(pd.DataFrame(before_attack_result1, index=model_names),4)

,0,1,2,3,4
nn_tanh,0.6381,0.4987,0.5027,0.9,0.5094
nn_relu_5_2,0.6420,0.5201,0.5025,0.9,0.5113
nn_relu_50_25,0.6650,0.5203,0.5075,0.9,0.5121
nn_relu_200_100,0.5855,0.5121,0.4979,0.8,0.5081
nn_relu_25_10,0.6052,0.5038,0.5039,0.7,0.5101
nn_log,0.6110,0.5210,0.5006,0.8,0.5020
nn_identity,0.5772,0.5220,0.4938,0.7,0.5021
tree_gini,0.4970,0.5019,0.5004,0.7,0.5290
tree_entropy,0.5397,0.5208,0.5039,0.7,0.5374
rf_gini,0.5263,0.5172,0.4988,0.8,0.5178


In [323]:
round(pd.DataFrame(after_attack_result1, index=model_names),4)

,0,1,2,3,4
nn_tanh,0.6530,0.5144,0.5029,0.8,0.4948
nn_relu_5_2,0.6327,0.5093,0.5016,0.8,0.5025
nn_relu_50_25,0.6585,0.5146,0.5077,0.8,0.5030
nn_relu_200_100,0.5994,0.5238,0.4971,0.8,0.4917
nn_relu_25_10,0.6170,0.5190,0.5038,0.8,0.4970
nn_log,0.6091,0.5213,0.5005,0.8,0.5011
nn_identity,0.5526,0.5091,0.4929,0.8,0.4951
tree_gini,0.4748,0.5061,0.4961,0.4,0.4681
tree_entropy,0.5292,0.5125,0.4907,0.5,0.5182
rf_gini,0.5207,0.5148,0.4956,0.4,0.4866


In [308]:
after_attack_result2 = np.array(after_attack_result)[[0,2,3,6,10,11,12,13]]
before_attack_result2 = np.array(before_attack_result)[[0,2,3,6,10,11,12,13]]

round(pd.DataFrame(after_attack_result2-before_attack_result2),4)

,0,1,2,3,4
0,-0.0052,-0.0042,-0.0008,-0.1,-0.0238
1,0.0065,0.0092,-0.0016,-0.3,-0.0293
2,-0.0075,0.0064,-0.0012,0.0,-0.0275
3,0.0031,-0.0046,-0.0000,-0.3,-0.0227
4,-0.0010,-0.0115,-0.0015,-0.1,-0.0362
5,0.0086,-0.0060,-0.0036,0.0,-0.0039
6,0.0178,-0.0039,0.0008,-0.1,-0.0002
7,-0.0004,-0.0018,-0.0012,-0.3,-0.0201


In [309]:
round(pd.DataFrame(before_attack_result2),4)

,0,1,2,3,4
0,0.5163,0.5068,0.4993,0.6,0.5027
1,0.6369,0.5075,0.5031,0.9,0.5103
2,0.2208,0.5080,0.4885,0.2,0.4931
3,0.5931,0.5148,0.4966,0.8,0.5054
4,0.5411,0.5190,0.5014,0.7,0.5163
5,0.5668,0.5075,0.5024,0.5,0.5043
6,0.5600,0.5132,0.5000,0.7,0.5001
7,0.6058,0.5164,0.4984,0.8,0.5056


In [310]:
round(pd.DataFrame(after_attack_result2),4)

,0,1,2,3,4
0,0.5111,0.5026,0.4985,0.5,0.4789
1,0.6434,0.5167,0.5015,0.6,0.4810
2,0.2132,0.5144,0.4873,0.2,0.4656
3,0.5962,0.5102,0.4966,0.5,0.4827
4,0.5401,0.5075,0.4998,0.6,0.4801
5,0.5753,0.5015,0.4988,0.5,0.5003
6,0.5778,0.5093,0.5009,0.6,0.4999
7,0.6054,0.5146,0.4972,0.5,0.4855


In [420]:
from sklearn.metrics import f1_score, roc_auc_score

final_results = []

for seed in [1,101,123,726,124,333,245]:
    
    print(seed)
    after_attack_result = []
    
    nn_tanh = MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), max_iter=1000, random_state=seed, warm_start=True)
    nn_relu_5_2 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=seed, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_relu_50_25 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(50, 25), random_state=seed, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_relu_200_100 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(200, 100), max_iter=2000, random_state=1, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_relu_25_10 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(25, 10), random_state=seed, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_log = MLPClassifier(activation = 'logistic', solver='adam', 
                        alpha=1e-4, hidden_layer_sizes=(5, 2),
                        learning_rate  = 'invscaling', 
                        random_state=seed, warm_start = True)
    nn_identity = MLPClassifier(activation = 'identity', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=seed, warm_start = True)

    tree_gini = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 0.05, min_samples_leaf = 0.001, max_features = None,random_state=seed)
    tree_entropy = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 0.05, min_samples_leaf = 0.001,random_state=seed)

    rf_gini = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001,random_state=seed)
    rf_entropy = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001,random_state=seed)

    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=seed)

    ada = AdaBoostClassifier(n_estimators=100)                     
    log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = True, solver = 'liblinear')


    mia_models = [nn_tanh,nn_relu_5_2, nn_relu_50_25, nn_relu_200_100, nn_relu_25_10, nn_log, nn_identity,
              tree_gini, tree_entropy, 
              rf_gini, rf_entropy, 
              gb, ada, 
              log_reg]

    i = 0
    print(model_names[i])

    for ba_model in mia_models:

        print(ba_model)

        sample_idx = np.random.choice(np.arange(len(x_tr)), len(x_te))

        prob_train = models[i].predict_proba(x_tr[sample_idx])
        prob_test = models[i].predict_proba(x_te)
        prob_target = models[i].predict_proba(x_ta)

        loss_train = cross_entropy(y_tr[sample_idx], prob_train)
        loss_test = cross_entropy(y_te, prob_test)
        loss_target = cross_entropy(y_ta, prob_target)

        feature_train = np.c_[prob_train, loss_train]
        feature_test = np.c_[prob_test, loss_test]
        feature_target = np.c_[prob_target, loss_target]

        x_mia = np.r_[feature_train, feature_test]
        y_mia = np.r_[labels_train[sample_idx], labels_test]

        x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

        ba_model.fit(x_mia_tr, y_mia_tr)

        pred1 = ba_model.predict(x_mia_te)
        print(np.sum(pred1)/len(pred1))
        prob = ba_model.predict_proba(x_mia_te)
        mia_conf1 = prob[np.arange(len(pred1)), pred1.astype(int)].mean()
        mia_auc1 = roc_auc_score(y_mia_te, prob[:,1])
        mia_f1_1 = f1_score(y_mia_te, pred1)
        mia_acc1 = accuracy_score(y_mia_te, pred1)

        pred_t = ba_model.predict(feature_target[tidx*10:(tidx+1)*10])
        prob = ba_model.predict_proba(feature_target[tidx*10:(tidx+1)*10])
        ta_mia_conf1 = prob[np.arange(len(pred_t)), pred_t.astype(int)].mean()
        ta_mia_acc1 = accuracy_score(labels_target[tidx*10:(tidx+1)*10], pred_t)


        #=========================================================================

        prob_train = poisoned_models[i].predict_proba(x_tr[sample_idx])
        prob_test = poisoned_models[i].predict_proba(x_te)
        prob_target = poisoned_models[i].predict_proba(x_ta)

        loss_train = cross_entropy(y_tr[sample_idx], prob_train)
        loss_test = cross_entropy(y_te, prob_test)
        loss_target = cross_entropy(y_ta, prob_target)

        feature_train = np.c_[prob_train, loss_train]
        feature_test = np.c_[prob_test, loss_test]
        feature_target = np.c_[prob_target, loss_target]

        x_mia = np.r_[feature_train, feature_test]
        y_mia = np.r_[labels_train[sample_idx], labels_test]

        x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

        pred = ba_model.predict(x_mia_te)
        print(np.sum(pred)/len(pred))
        prob = ba_model.predict_proba(x_mia_te)
        mia_conf2 = prob[np.arange(len(pred1)), pred1.astype(int)].mean()
        mia_auc2 = roc_auc_score(y_mia_te, prob[:,1])
        mia_f1_2 = f1_score(y_mia_te, pred)
        mia_acc2 = accuracy_score(y_mia_te, pred)

        pred = ba_model.predict(feature_target[tidx*10:(tidx+1)*10])
        prob = ba_model.predict_proba(feature_target[tidx*10:(tidx+1)*10])
        ta_mia_conf2 = prob[np.arange(len(pred_t)), pred_t.astype(int)].mean()
        ta_mia_acc2 = accuracy_score(labels_target[tidx*10:(tidx+1)*10], pred)

        after_attack_result.append([mia_auc1, mia_f1_1, ta_mia_conf1, ta_mia_acc1, mia_auc2, mia_f1_2, ta_mia_conf2, ta_mia_acc2])
        print(after_attack_result[-1])
    
    final_results.append(np.array(after_attack_result))
    
        
    

1
nn_tanh
MLPClassifier(activation='tanh', alpha=0.1, hidden_layer_sizes=(10, 2),
              max_iter=1000, random_state=1, solver='lbfgs', warm_start=True)
0.854590482929418
0.8521139218114275
[0.5151881389333662, 0.6408883082952318, 0.505624523895286, 0.9, 0.5273661545440276, 0.642595499738357, 0.4956857833707044, 0.8]
MLPClassifier(alpha=0.1, hidden_layer_sizes=(5, 2), learning_rate='invscaling',
              random_state=1, warm_start=True)
0.0
0.0
[0.5, 0.0, 0.5013017549021608, 0.1, 0.5, 0.0, 0.5013017549021608, 0.1]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(50, 25),
              learning_rate='invscaling', random_state=1, warm_start=True)
0.14399433928887317
0.13391119759419778
[0.5086277605090566, 0.22821350762527232, 0.5084032737303389, 0.2, 0.5132273790814911, 0.21650650084793668, 0.5156806794312265, 0.2]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(200, 100),
              learning_rate='invscaling', max_iter=2000, random_state=1,
              warm_start=True)
0.

0.6623032018397311
0.6396603573323899
[0.5055644975741385, 0.566758325695081, 0.5071123938114647, 0.7, 0.5210406143793304, 0.5748558965570961, 0.508394801504651, 0.6]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(50, 25),
              learning_rate='invscaling', random_state=123, warm_start=True)
0.7277551742437643
0.7160799575446666
[0.5182324758646034, 0.6036321706543673, 0.5110909167546397, 0.8, 0.5290060477384514, 0.6093252244425138, 0.5151435792159091, 0.7]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(200, 100),
              learning_rate='invscaling', max_iter=2000, random_state=1,
              warm_start=True)
0.47992216522200604
0.507164337519901
[0.5164738565969786, 0.49874326750448833, 0.5071386681103467, 0.6, 0.5344892953893572, 0.5198440822111977, 0.5153749998658912, 0.5]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(25, 10),
              learning_rate='invscaling', random_state=123, warm_start=True)
0.13638775871218822
0.12913497258093048
[0.5318096902592944, 0.242

0.11551388643198301
0.10843799752343888
[0.5008420582287523, 0.18859401468097123, 0.506488668473868, 0.2, 0.5258260133562045, 0.182717486179808, 0.5185291169921779, 0.2]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(200, 100),
              learning_rate='invscaling', max_iter=2000, random_state=1,
              warm_start=True)
0.6398372545551034
0.6446134795683708
[0.509327099411032, 0.5631668470697387, 0.5093917572700637, 0.7, 0.5041495310956625, 0.5623251476530929, 0.5113663955721939, 0.6]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(25, 10),
              learning_rate='invscaling', random_state=124, warm_start=True)
0.6831770741199363
0.6697328851937024
[0.5210203979445582, 0.5920738974970203, 0.5082828327802864, 0.7, 0.5261158252132111, 0.5895375284306293, 0.512524877271907, 0.6]
MLPClassifier(activation='logistic', hidden_layer_sizes=(5, 2),
              learning_rate='invscaling', random_state=124, warm_start=True)
0.0
0.0
[0.525447483442349, 0.0, 0.5024187944136183, 0.1, 

0.5697859543605165
0.5283920042455333
[0.5095208234642545, 0.5369944097336402, 0.5020033803123838, 0.6, 0.5002195496303581, 0.5136856601824755, 0.49000011248163594, 0.5]
MLPClassifier(alpha=0.001, hidden_layer_sizes=(25, 10),
              learning_rate='invscaling', random_state=245, warm_start=True)
0.1464709004068636
0.1247125420130904
[0.5193868135940368, 0.23666395884105065, 0.5062543384703257, 0.2, 0.517537103532393, 0.19795221843003413, 0.512933655757456, 0.2]
MLPClassifier(activation='logistic', hidden_layer_sizes=(5, 2),
              learning_rate='invscaling', random_state=245, warm_start=True)
0.7985140633292057
0.7963912966566425
[0.5100992690262712, 0.6114980199371842, 0.5010646356729603, 0.8, 0.5120691144546186, 0.6214198452558707, 0.5008943923996286, 0.7]
MLPClassifier(activation='identity', alpha=0.1, hidden_layer_sizes=(5, 2),
              random_state=245, warm_start=True)
0.7541128604280912
0.7518131965328144
[0.5173328996853548, 0.6019555051721696, 0.5074603390692

array([0.50539258, 0.50539451, 0.50538311, 0.50253874, 0.50479475,
       0.50525148, 0.50489393, 0.54956138, 0.54259737, 0.50523417])

In [421]:
final_results = np.array(final_results)
mean_result = np.mean(final_results,axis=0)
pdresult = pd.DataFrame(mean_result, index=model_names)
pdresult.columns = ['auc1', 'f1 1', 'conf1', 'acc1', 'auc2', 'f1 2', 'conf2', 'acc2']
pdresult['decrease'] = mean_result[:,7]-mean_result[:,3]
round(pdresult[pdresult['f1 1']>0.5],4)

,auc1,f1 1,conf1,acc1,auc2,f1 2,conf2,acc2,decrease
nn_tanh,0.5136,0.5593,0.5075,0.6857,0.5226,0.5568,0.5081,0.5714,-0.1143
nn_relu_200_100,0.5151,0.5695,0.5087,0.7143,0.5162,0.5686,0.5101,0.6143,-0.1000
nn_identity,0.5184,0.5598,0.5097,0.7429,0.5211,0.5663,0.5123,0.6571,-0.0857
tree_gini,0.5138,0.5076,0.5650,0.4857,0.5148,0.4996,0.5056,0.4714,-0.0143
tree_entropy,0.5185,0.5203,0.5677,0.6000,0.5138,0.5085,0.4923,0.5143,-0.0857
rf_gini,0.5191,0.5237,0.5321,0.6143,0.5107,0.5122,0.5104,0.4714,-0.1429
rf_entropy,0.5184,0.5203,0.5301,0.5714,0.5155,0.5066,0.5128,0.4571,-0.1143
gb,0.5138,0.5287,0.5214,0.5714,0.5136,0.5252,0.5002,0.4714,-0.1000
ada,0.5129,0.5448,0.5002,0.6143,0.5138,0.5392,0.5000,0.6286,0.0143
log_reg,0.5230,0.6147,0.5080,0.8143,0.5245,0.6176,0.5113,0.7000,-0.1143


In [422]:
bools = final_results[:,:,0]>0.5
np.array(model_names)[np.sum(bools,axis=0)>5]

array(['nn_tanh', 'nn_relu_5_2', 'nn_relu_50_25', 'nn_relu_200_100',
       'nn_relu_25_10', 'nn_log', 'nn_identity', 'tree_gini',
       'tree_entropy', 'rf_gini', 'rf_entropy', 'gb', 'ada', 'log_reg'],
      dtype='<U15')

In [423]:
bools = final_results[:,:,0]>0.5
weighted_result = np.zeros((14,8))

for i in range(14):
    weighted = final_results[:,i][bools[:,i]]
    weighted_result[i] = np.mean(weighted, axis=0)

In [424]:
pdresult = pd.DataFrame(weighted_result, index=model_names)
pdresult.columns = ['auc1', 'f1 1', 'conf1', 'acc1', 'auc2', 'f1 2', 'conf2', 'acc2']
pdresult['decrease'] = weighted_result[:,7]-weighted_result[:,3]
round(pdresult[np.sum(bools,axis=0)>5],4)

,auc1,f1 1,conf1,acc1,auc2,f1 2,conf2,acc2,decrease
nn_tanh,0.5136,0.5593,0.5075,0.6857,0.5226,0.5568,0.5081,0.5714,-0.1143
nn_relu_5_2,0.5117,0.5775,0.5067,0.7500,0.5158,0.5725,0.5045,0.6667,-0.0833
nn_relu_50_25,0.5158,0.3909,0.5088,0.4286,0.5202,0.3879,0.5162,0.4000,-0.0286
nn_relu_200_100,0.5151,0.5695,0.5087,0.7143,0.5162,0.5686,0.5101,0.6143,-0.1000
nn_relu_25_10,0.5195,0.4905,0.5072,0.5714,0.5225,0.4860,0.5113,0.5143,-0.0571
nn_log,0.5166,0.4301,0.5023,0.6000,0.5200,0.4318,0.5026,0.5286,-0.0714
nn_identity,0.5184,0.5598,0.5097,0.7429,0.5211,0.5663,0.5123,0.6571,-0.0857
tree_gini,0.5138,0.5076,0.5650,0.4857,0.5148,0.4996,0.5056,0.4714,-0.0143
tree_entropy,0.5185,0.5203,0.5677,0.6000,0.5138,0.5085,0.4923,0.5143,-0.0857
rf_gini,0.5191,0.5237,0.5321,0.6143,0.5107,0.5122,0.5104,0.4714,-0.1429


In [316]:
from sklearn.metrics import f1_score

before_attack_result = []
after_attack_result = []

for i in [0]:
    
    print(model_names[i])
    
    sample_idx = np.random.choice(np.arange(len(x_tr)), len(x_te))
    
    prob_train = models[i].predict_proba(x_tr[sample_idx])
    prob_test = models[i].predict_proba(x_te)
    prob_target = models[i].predict_proba(x_ta)

    loss_train = cross_entropy(y_tr[sample_idx], prob_train)
    loss_test = cross_entropy(y_te, prob_test)
    loss_target = cross_entropy(y_ta, prob_target)

    feature_train = np.c_[prob_train, loss_train]
    feature_test = np.c_[prob_test, loss_test]
    feature_target = np.c_[prob_target, loss_target]

    x_mia = np.r_[feature_train, feature_test]
    y_mia = np.r_[labels_train[sample_idx], labels_test]

    x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)
    
#     a_idxs = delete(x_mia_tr, feature_target)
#     x_mia_tr = x_mia_tr[a_idxs]
#     y_mia_tr = y_mia_tr[a_idxs]
    
    x_mia_ta, y_mia_ta = feature_target[tidx*10:(tidx+1)*10], labels_target[tidx*10:(tidx+1)*10]
    before_attack_result = MIA_attack(x_mia_tr, y_mia_tr, x_mia_te, y_mia_te, x_mia_ta, y_mia_ta)    
    #before_attack_result.append([mia_f1_1, mia_acc1, mia_conf1, ta_mia_acc1, ta_mia_conf1])
    print(before_attack_result)
    

    #=========================================================================
    
    prob_train = poisoned_models[i].predict_proba(x_tr[sample_idx])
    prob_test = poisoned_models[i].predict_proba(x_te)
    prob_target = poisoned_models[i].predict_proba(x_ta)

    loss_train = cross_entropy(y_tr[sample_idx], prob_train)
    loss_test = cross_entropy(y_te, prob_test)
    loss_target = cross_entropy(y_ta, prob_target)

    feature_train = np.c_[prob_train, loss_train]
    feature_test = np.c_[prob_test, loss_test]
    feature_target = np.c_[prob_target, loss_target]

    x_mia = np.r_[feature_train, feature_test]
    y_mia = np.r_[labels_train[sample_idx], labels_test]

    x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)
    
#     a_idxs = delete(x_mia_tr, feature_target)
#     x_mia_tr = x_mia_tr[a_idxs]
#     y_mia_tr = y_mia_tr[a_idxs]
    
    x_mia_ta, y_mia_ta = feature_target[tidx*10:(tidx+1)*10], labels_target[tidx*10:(tidx+1)*10]
    after_attack_result = MIA_attack(x_mia_tr, y_mia_tr, x_mia_te, y_mia_te, x_mia_ta, y_mia_ta)
    #after_attack_result.append([mia_f1_2, mia_acc2, mia_conf2, ta_mia_acc2, ta_mia_conf2])
    print(after_attack_result)
    
    print(round(pd.DataFrame(after_attack_result-before_attack_result),4))
    

nn_tanh
[[0.85653635 0.64239272 0.51353264 0.49817128 1.         0.94736842
  0.9        0.50373503]
 [0.         0.         0.49619671 0.49899125 0.         0.
  0.1        0.49899125]
 [0.16451442 0.25780836 0.50398019 0.49424751 0.1        0.2
  0.2        0.49844858]
 [0.90076066 0.65465995 0.51494782 0.50451448 1.         0.94736842
  0.9        0.5098259 ]
 [0.18397311 0.27417695 0.50079604 0.49393059 0.2        0.36363636
  0.3        0.50111813]
 [0.063683   0.12157107 0.50150363 0.49833953 0.         0.
  0.1        0.49943059]
 [0.55687246 0.54336224 0.5156554  0.49404828 0.8        0.82352941
  0.7        0.50101245]
 [0.58252255 0.55495848 0.51653989 0.49880672 0.8        0.94117647
  0.9        0.51803084]
 [0.58252255 0.55528416 0.51689368 0.49883977 0.8        0.94117647
  0.9        0.51803084]
 [0.52662303 0.52497854 0.51052538 0.49846723 0.7        0.875
  0.8        0.51267716]
 [0.5225544  0.52499138 0.51247125 0.49824793 0.8        0.94117647
  0.9        0.5142605

In [249]:
def MIA_attack(xtr, ytr, xte, yte, xta, yta):
    
    nn_tanh = MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), max_iter=1000, random_state=1, warm_start=True)
    nn_relu_5_2 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_relu_50_25 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(50, 25), random_state=1, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_relu_200_100 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(200, 100), max_iter=2000, random_state=1, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_relu_25_10 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(25, 10), random_state=1, learning_rate  = 'invscaling',
                        warm_start = True)
    nn_log = MLPClassifier(activation = 'logistic', solver='adam', 
                        alpha=1e-4, hidden_layer_sizes=(5, 2),
                        learning_rate  = 'invscaling', 
                        random_state=1, warm_start = True)
    nn_identity = MLPClassifier(activation = 'identity', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, warm_start = True)

    tree_gini = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 0.05, min_samples_leaf = 0.001, max_features = None)
    tree_entropy = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 0.05, min_samples_leaf = 0.001)

    rf_gini = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)
    rf_entropy = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)

    gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

    ada = AdaBoostClassifier(n_estimators=100)                     
    log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = True, solver = 'liblinear')


    mia_models = [nn_tanh,nn_relu_5_2, nn_relu_50_25, nn_relu_200_100, nn_relu_25_10, nn_log, nn_identity,
              tree_gini, tree_entropy, 
              rf_gini, rf_entropy, 
              gb, ada, 
              log_reg]

    att_result = []
    
    def eval(m, x, y):
        pred = m.predict(x)
        prob = m.predict_proba(x)
        mia_conf = prob[:,1].mean()
        mia_f1 = f1_score(y, pred)
        mia_acc = accuracy_score(y, pred)
        return [np.sum(pred)/len(pred), mia_f1, mia_acc, mia_conf]

    #print(xtr.shape, ytr.shape)
    for m in mia_models:
        m.fit(xtr, ytr)
        te_result = eval(m, xte, yte)
        ta_result = eval(m, xta, yta)
        
        att_result.append(te_result+ta_result)
    return np.array(att_result)

In [226]:
nn_tanh = MLPClassifier(activation = 'tanh', solver='lbfgs', alpha=1e-1, hidden_layer_sizes=(10, 2), max_iter=1000, random_state=1, warm_start=True)
nn_relu_5_2 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_50_25 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(50, 25), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_200_100 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(200, 100), max_iter=2000, random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_relu_25_10 = MLPClassifier(activation = 'relu', solver='adam', alpha=1e-3, hidden_layer_sizes=(25, 10), random_state=1, learning_rate  = 'invscaling',
                    warm_start = True)
nn_log = MLPClassifier(activation = 'logistic', solver='adam', 
                    alpha=1e-4, hidden_layer_sizes=(5, 2),
                    learning_rate  = 'invscaling', 
                    random_state=1, warm_start = True)
nn_identity = MLPClassifier(activation = 'identity', solver='adam', alpha=1e-1, hidden_layer_sizes=(5, 2), random_state=1, warm_start = True)

tree_gini = DecisionTreeClassifier(criterion = 'gini', min_samples_split = 0.05, min_samples_leaf = 0.001, max_features = None)
tree_entropy = DecisionTreeClassifier(criterion = 'entropy', min_samples_split = 0.05, min_samples_leaf = 0.001)

rf_gini = RandomForestClassifier(n_estimators=100, criterion = 'gini', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)
rf_entropy = RandomForestClassifier(n_estimators=100, criterion = 'entropy', max_features = None,  min_samples_split = 0.05, min_samples_leaf = 0.001)

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

ada = AdaBoostClassifier(n_estimators=100)                     
log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = True, solver = 'liblinear')


mia_models = [nn_tanh,nn_relu_5_2, nn_relu_50_25, nn_relu_200_100, nn_relu_25_10, nn_log, nn_identity,
          tree_gini, tree_entropy, 
          rf_gini, rf_entropy, 
          gb, ada, 
          log_reg]

att_result = []


for i in range(len(models)):
    print(model_names[i])
    prob_train = models[i].predict_proba(x_tr[sample_idx])
    prob_test = models[i].predict_proba(x_te)
    prob_target = models[i].predict_proba(x_ta)

    loss_train = cross_entropy(y_tr[sample_idx], prob_train)
    loss_test = cross_entropy(y_te, prob_test)
    loss_target = cross_entropy(y_ta, prob_target)

    feature_train = np.c_[prob_train, loss_train]
    feature_test = np.c_[prob_test, loss_test]
    feature_target = np.c_[prob_target, loss_target]

    x_mia = np.r_[feature_train, feature_test]
    y_mia = np.r_[labels_train[sample_idx], labels_test]

    x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)


    def test(m):
        m.fit(x_mia_tr, y_mia_tr)
        pred = m.predict(x_mia_te)
        prob = m.predict_proba(x_mia_te)
        mia_conf1 = prob[:,1].mean()
        mia_f1_1 = f1_score(y_mia_te, pred)
        mia_acc1 = accuracy_score(y_mia_te, pred)
        return [np.sum(pred)/len(pred), mia_f1_1, mia_acc1]

    for m in mia_models:
        att_result.append(test(m))
        print(test(m))

nn_tanh
[0.6018043516716788, 0.564597846738442, 0.5135326375375907]
[0.0, 0.0, 0.4845214930125597]
[0.12860428091278966, 0.21752265861027192, 0.49601981248894395]
[0.48045285689014683, 0.5055062166962699, 0.5075181319653281]
[0.1415177781708827, 0.2358642972536349, 0.49796568193879354]
[0.0, 0.0, 0.4845214930125597]
[0.5883601627454449, 0.5612179487179487, 0.5156554042101539]
[0.4196002122766673, 0.47370412410139984, 0.5078719264107554]
[0.3794445427206793, 0.44989128286222574, 0.5076950291880418]
[0.4993808597205024, 0.5281506013595957, 0.5211392181142757]
[0.4963736069343711, 0.5237762237762239, 0.5181319653281443]
[0.3916504510879179, 0.4574882995319813, 0.5078719264107554]
[0.41535467893154077, 0.47130368681109847, 0.5078719264107554]
[0.6375375906598266, 0.5814667075790119, 0.5174243764372899]
nn_relu_5_2
[0.6366531045462587, 0.5714285714285715, 0.5133557403148771]
[0.0, 0.0, 0.5011498319476384]
[0.47673801521316117, 0.4928377153218495, 0.5052184680700513]
[0.5662480099062445, 0.5

[0.6076419600212276, 0.542705484960592, 0.49708119582522553]
[0.8712188218644967, 0.6305955624756714, 0.4963736069343711]
rf_entropy
[0.6269237572970104, 0.5755440739001096, 0.5204316292234212]
[1.0, 0.6692561205273069, 0.5029188041747744]
[0.10065451972404034, 0.1735052754982415, 0.5011498319476384]
[0.3021404563948346, 0.3884860470226324, 0.5076950291880418]
[0.10348487528745799, 0.17911318553092181, 0.50221121528392]
[0.3955421899876172, 0.43473124630832843, 0.49212807358924465]
[0.0, 0.0, 0.49708119582522553]
[0.4638245179550681, 0.5028362305580971, 0.5193702458871395]
[0.501326729170352, 0.5231636427690681, 0.5211392181142757]
[0.479037679108438, 0.5101783462439201, 0.5190164514417124]
[0.48080665133557404, 0.5117784571120301, 0.5197240403325668]
[0.43923580399787726, 0.4791588434096883, 0.5092871041924641]
[0.4323368123120467, 0.4743710989218839, 0.5084026180788962]
[0.5034494958429153, 0.5122165582703463, 0.5091102069697506]
gb
[0.6301079073058553, 0.5620813210780495, 0.50274190

In [238]:
att_result = np.array(att_result)
avg = np.zeros((14,3))

for i in range(14):
    avg += att_result[i*14:(i+1)*14]/14
    #avg[i] = att_result[i*14:(i+1)*14].mean(axis=0)
    
avg 

array([[0.68754896, 0.59129476, 0.51636299],
       [0.35714286, 0.23758099, 0.49643678],
       [0.58034925, 0.48531615, 0.51085391],
       [0.6053423 , 0.5133809 , 0.51201638],
       [0.61236764, 0.51531488, 0.51335574],
       [0.75058755, 0.57720327, 0.51008314],
       [0.67559576, 0.53651589, 0.51209219],
       [0.56826969, 0.5436351 , 0.51537742],
       [0.56458012, 0.54162681, 0.51542796],
       [0.54993556, 0.53800158, 0.5179677 ],
       [0.53417907, 0.53003458, 0.51653989],
       [0.57871926, 0.54246126, 0.51352   ],
       [0.5661722 , 0.53599356, 0.51310303],
       [0.64648354, 0.55735788, 0.51143514]])

In [233]:
avg.mean(axis=0)

array([0.59123384, 0.51755126, 0.51246945])

In [212]:
np.sum(ba_model.predict(x_mia_te))/len(ba_model.predict(x_mia_te))

0.8690960551919334

In [210]:
y_mia

array([1., 1., 1., ..., 0., 0., 0.])

In [189]:
feature_target[tidx*10:(tidx+1)*10]

array([[9.93622553e-01, 6.37744685e-03, 6.39786964e-03],
       [9.96298378e-01, 3.70162227e-03, 3.70849023e-03],
       [9.94690924e-01, 5.30907552e-03, 5.32321874e-03],
       [8.27599006e-01, 1.72400994e-01, 1.89226535e-01],
       [9.16819945e-01, 8.31800551e-02, 8.68441784e-02],
       [8.76894657e-01, 1.23105343e-01, 1.31368411e-01],
       [9.90793000e-01, 9.20699997e-03, 9.24964636e-03],
       [2.63913997e-02, 9.73608600e-01, 3.63471709e+00],
       [2.09591275e-03, 9.97904087e-01, 2.09811225e-03],
       [9.90870526e-01, 9.12947384e-03, 9.17140287e-03]])

In [190]:
prob_target = models[i].predict_proba(x_ta)
loss_target = cross_entropy(y_ta, prob_target)
feature_target = np.c_[prob_target, loss_target]
feature_target[tidx*10:(tidx+1)*10]

array([[9.98096287e-01, 1.90371338e-03, 1.90552775e-03],
       [9.99951742e-01, 4.82579239e-05, 4.82590883e-05],
       [9.97010440e-01, 2.98956031e-03, 2.99403797e-03],
       [8.08977878e-01, 1.91022122e-01, 2.11983707e-01],
       [9.30176673e-01, 6.98233271e-02, 7.23807400e-02],
       [9.63565954e-01, 3.64340457e-02, 3.71143406e-02],
       [9.24668331e-01, 7.53316687e-02, 7.83201666e-02],
       [6.92854975e-01, 3.07145025e-01, 3.66934573e-01],
       [2.88301376e-01, 7.11698624e-01, 3.40100738e-01],
       [9.98884688e-01, 1.11531153e-03, 1.11593395e-03]])

In [195]:
round(pd.DataFrame(after_attack_result, index=model_names, columns=['mia acc', 'mia conf', 'ta mia acc', 'ta mia conf']),4)

,mia acc,mia conf,ta mia acc,ta mia conf
nn_tanh,0.7448,0.7620,0.9,0.7417
nn_relu_5_2,0.7498,0.7337,0.9,0.7061
nn_relu_50_25,0.7592,0.7517,0.8,0.7148
nn_relu_200_100,0.7792,0.7268,0.8,0.6895
nn_relu_25_10,0.7507,0.7422,0.8,0.6997
nn_log,0.7515,0.7567,0.9,0.7438
nn_identity,0.7498,0.7468,0.9,0.7425
tree_gini,0.7463,0.7539,0.9,0.7568
tree_entropy,0.7458,0.7639,0.9,0.7649
rf_gini,0.7529,0.7452,0.9,0.7431


In [155]:
# ti = target_dataset[1][:,1][tidx*10:(tidx+1)*10]
# tmp1 = np.concatenate(([1-prob1], [prob1]), axis=0).T
# tmp2 = np.concatenate(([1-prob2], [prob2]), axis=0).T
# np.mean(tmp2[np.arange(len(ti)), ti]-tmp1[np.arange(len(ti)), ti])

In [196]:
after = round(pd.DataFrame(poisoned_result, index = model_names, columns = ['train acc', 'test acc', 'target acc', 'confidence']),4)
after

,train acc,test acc,target acc,confidence
nn_tanh,-0.0106,-0.0074,-0.3,-0.2082
nn_relu_5_2,-0.0131,-0.0187,-0.6,-0.3932
nn_relu_50_25,0.0039,-0.0047,0.0,-0.0419
nn_relu_200_100,-0.0010,-0.0097,0.0,-0.0182
nn_relu_25_10,-0.0009,-0.0043,0.0,-0.1291
nn_log,-0.0090,-0.0110,-0.4,-0.2707
nn_identity,-0.0431,-0.0453,-0.5,-0.3474
tree_gini,-0.0332,-0.0364,-0.3,-0.1991
tree_entropy,-0.0215,-0.0220,-0.2,-0.1377
rf_gini,-0.0357,-0.0383,-0.3,-0.2382


In [197]:
after.mean(axis=0) 

train acc    -0.021479
test acc     -0.023607
target acc   -0.242857
confidence   -0.188329
dtype: float64

In [198]:
after.std(axis=0) 

train acc     0.016653
test acc      0.015338
target acc    0.213809
confidence    0.122475
dtype: float64